In [1]:
import sys,os,subprocess,glob
import pandas as pd
import numpy as np
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio import SeqIO
import urllib.request
sys.path.append('../')
from pyamrfinder import tools, app

In [2]:
def fetch_sequence_db(name='card'):
    """get sequences"""
    
    path = '../pyamrfinder/data/'
    if name == 'card':
        url = 'https://github.com/tseemann/abricate/raw/master/db/card/sequences'
    elif name == 'resfinder':
        url = 'https://raw.githubusercontent.com/tseemann/abricate/master/db/resfinder/sequences'
    filename = os.path.join(path,"%s.fa" %name)
    if not os.path.exists(filename):
        urllib.request.urlretrieve(url, filename)
    return

fetch_sequence_db('resfinder')

In [ ]:
fname='/local/abricate/db/card/sequences'
f=open(fname,'r')
for i in f.readlines():

sp=SeqIO.parse(handle=f,format='fasta')
seqs=[]
print (seqs)

In [6]:
def make_blast_database(filenames):
    """Make blast dbs of multiple input files"""

    rec=[]
    for n in filenames:
        seqs = list(SeqIO.parse(n,'fasta'))
        for s in seqs:
            s.id = n + '~' + s.id
        rec.extend(seqs)
    #ref = list(SeqIO.parse('genomes/ecoli_k12.fa','fasta'))
    #ref[0].id = 'ecoli_k12~1'
    #rec.extend(ref)
    SeqIO.write(rec, 'out.fasta', 'fasta')
    cmd = 'makeblastdb -dbtype nucl -in out.fasta'
    subprocess.check_output(cmd, shell=True)
    return

make_blast_database(['../test_files/RF15B.fa'])

In [4]:
def run_blast(db='card', ident=90, coverage=.75):
    """blast card seqs"""

    path = '../pyamrfinder/data/%s.fa' %db
    dbseqs = list(SeqIO.parse(path,'fasta'))  
    bl = tools.blast_sequences('scaffolds.fasta', dbseqs, maxseqs=100, evalue=.1,
                               cmd='blastn', show_cmd=True)
    #print (bl[:5])
    bl['qlength'] = bl.sequence.str.len()
    bl['coverage'] = bl.length/bl.qlength
    bl = bl[bl.coverage>coverage]
    bl = bl[bl.pident>ident]
    bl['id'] = bl.sseqid.apply(lambda x: x.split('~')[0],1)
    bl['contig'] = bl.sseqid.apply(lambda x: x.split('~')[1],1)
    bl['gene'] = bl['qseqid'].apply(lambda x: x.split('~~~')[1],1)
    #print (bl)
    cols = ['qseqid','sseqid','pident','sstart','send','coverage','contig','gene','id']
    bl = bl[cols]
    return bl

bl = run_blast('resfinder')

blastn -out tempseq_blast.txt -outfmt "6 qseqid sseqid qseq sseq pident qcovs length mismatch gapopen qstart qend sstart send evalue bitscore stitle" -query tempseq.fa -db scaffolds.fasta -evalue 0.1 -max_target_seqs 100 -num_threads 2


In [26]:
bl[:3]

,qseqid,sseqid,pident,sstart,send,coverage,contig,gene,id
0,resfinder~~~aac(3)-IV_1~~~DQ241380~~~Gentamici...,../test_files/RF15B.fa~NODE_47_length_1709_cov...,100.000,1,629,0.809524,NODE_47_length_1709_cov_9.27813,aac(3)-IV_1,../test_files/RF15B.fa
2,resfinder~~~aac(3)-IVa_1~~~X01385~~~Gentamicin...,../test_files/RF15B.fa~NODE_47_length_1709_cov...,99.843,1,637,0.811705,NODE_47_length_1709_cov_9.27813,aac(3)-IVa_1,../test_files/RF15B.fa
4,resfinder~~~aadA11_2~~~AJ567827~~~Streptomycin,../test_files/RF15B.fa~NODE_25_length_25084_co...,92.785,18320,17534,0.997475,NODE_25_length_25084_cov_86.3603,aadA11_2,../test_files/RF15B.fa


In [92]:
def find_gene_hits(res, gene, filename, db='card'):
    """Get blast hit results"""

    path = '../pyamrfinder/data/%s.fa' %db
    #dbseqs = SeqIO.to_dict(SeqIO.parse(path,'fasta'))
    dbseqs = tools.fasta_to_dataframe(path)
    dbseqs['gene'] = dbseqs.description.apply(lambda x: x.split('~~~')[1],1)
    #print (dbseqs)
    x = res[res.gene==gene]
    found=[]
    hits = []
    for i,r in x.iterrows():
        name=r.id
        #if name not in isolates: continue
        seqs = SeqIO.to_dict(SeqIO.parse(filename,'fasta'))
        node = r.contig
        if r.sstart<r.send:
            s = seqs[node].seq[r.sstart:r.send]
        else:
            s = seqs[node].seq[r.send:r.sstart].reverse_complement()

        s = SeqRecord(id=name,seq=s)
        found.append(s)
        print (name, r.gene, r['coverage'], r['pident'], len(s), node)
        #add card seq
        hits.append(seqs[node])

    row = dbseqs[dbseqs.gene==gene].iloc[0]
    print (row)
    found.append(SeqRecord(id=row['name'],seq=Seq(row.sequence)))
    seqfile = 'temp.fa'
    SeqIO.write(found,seqfile,'fasta')
    SeqIO.write(hits,'contigs.fa','fasta')
    #maaft_alignment(seqfile)
    aln = tools.clustal_alignment(seqfile)
    print (aln)
    return


In [93]:
find_gene_hits(bl, 'aac(3)-IV_1', '../test_files/RF15B.fa', 'resfinder')

../test_files/RF15B.fa aac(3)-IV_1 0.8095238095238095 100.0 628 NODE_47_length_1709_cov_9.27813
name           resfinder~~~aac(3)-IV_1~~~DQ241380~~~Gentamici...
sequence       GTGCAATACGAATGGCGAAAAGCCGAGCTCATCGGTCAGCTTCTCA...
description    resfinder~~~aac(3)-IV_1~~~DQ241380~~~Gentamici...
type                                                         CDS
gene                                                 aac(3)-IV_1
Name: 70, dtype: object
SingleLetterAlphabet() alignment with 2 rows and 777 columns
--------------------------------------------...TGA ../test_files/RF15B.fa
GTGCAATACGAATGGCGAAAAGCCGAGCTCATCGGTCAGCTTCT...TGA resfinder~~~aac_3_-IV_1~~~DQ24
